# Project Crobat: Lead Generation using Google Maps API
## Introduction
## Welcome to Project Crobat !
<img src="crobat.jpg" alt="drawing" width="400"/>
<!-- ![image](crobat.jpg) -->


## Generating potential outlets

In [ ]:
match_by = 'brand'

In [ ]:
user = 'Zaki'

In [ ]:
country = 'sg'

In [ ]:
min_rating = '1.0'

In [ ]:
area_digits = '4'

In [ ]:
common_words = 'singapore, city, express, service, kitchen, food, republic'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import unicodedata
import re
from cleanco import basename
import requests
import json
from name_matching.name_matcher import NameMatcher

try:
    min_rating = float(min_rating)
    if min_rating < 0 or min_rating > 5:
        print("Minimum rating is not a valid number")
        print("Defaulting to minimum rating of 0")
        min_rating = 0
except:
    print("Minimum rating is not a valid number")
    print("Defaulting to minimum rating of 0")
    min_rating = 0

area_digits = int(area_digits)

In [ ]:
# Toggles between both api keys. This section can be deprecated in the future when consolidating to one api key
# Can also leverage Deepnote environment variables to hide api key: https://help.deepnote.com/en/articles/6645567-how-to-create-and-use-an-api-key

if user == 'Ben':
    api_key = 'placeholder' # ben_api_key
elif user == 'Zaki':
    api_key = 'placeholder' # zaki_api_key

In [ ]:
# Add relevant input fields above

In [ ]:
# Add helper functions

In [ ]:
# Add Google API Code Here

In [ ]:
# End Google API section by saving to df g_outlet_df_raw
g_outlet_df_raw = pd.read_csv('/work/datasets/combined_primary_data_with_postal_code_culmulative (cafe + restaraunt + bar).csv', dtype={'POSTALCODE_GOOGLE': object,
                                                                                            'POSTALCODE': object,
                                                                                            'POSTALCODE_COMBINED': object})

In [ ]:
g_outlet_df_raw.describe()

,rating,user_ratings_total,lat,long,price_level
count,25637.000000,25637.000000,25637.000000,25637.000000,5382.000000
mean,3.966708,210.147560,1.333500,103.845201,1.711817
std,0.807047,691.855981,0.043748,0.060569,0.595159
min,1.000000,1.000000,1.250185,103.623211,1.000000
25%,3.600000,6.000000,1.301144,103.817007,1.000000
50%,4.100000,30.000000,1.324716,103.848486,2.000000
75%,4.500000,169.000000,1.357985,103.883900,2.000000
max,5.000000,49569.000000,1.468364,104.004305,4.000000


In [ ]:
g_outlet_df_raw.head()

,business_status,name,place_id,rating,types,user_ratings_total,vicinity,Location,Type,lat,long,price_level,POSTALCODE_GOOGLE,POSTALCODE,POSTALCODE_COMBINED
0,OPERATIONAL,TRIFECTA BAR,ChIJ-6LoaOgZ2jER-h8WMod2d8g,3.7,"['bar', 'point_of_interest', 'establishment']",3.0,"10A Exeter Road, #01-02",111 Somerset,AreaSearch,1.299745,103.838952,NaN,239958,239958,239958
1,OPERATIONAL,Moon Rooftop Bar & Lounge Pte. Ltd.,ChIJ-6eVAvgZ2jERXmewPCfIXSU,4.6,"['bar', 'night_club', 'restaurant', 'food', 'p...",274.0,"70 Anson Road, #27-01 Hub Synergy Point",100 AM,AreaSearch,1.273924,103.844467,NaN,079911,079911,79911
2,OPERATIONAL,Bitters & Love,ChIJ-6zr_AoZ2jERoSRdI6lsE0s,4.6,"['bar', 'point_of_interest', 'establishment']",492.0,118 Telok Ayer Street,Amoy Street,AreaSearch,1.281972,103.848167,3.0,068587,68589,68587
3,OPERATIONAL,Muddy Murphy's Irish Pub,ChIJ-ZtvtowZ2jERyywaJQ_r4DU,4.4,"['bar', 'restaurant', 'food', 'point_of_intere...",1027.0,442 Orchard Road #01-02 to 05 Claymore Connect,Delfi Orchard,AreaSearch,1.307495,103.828947,2.0,238879,238879,238879
4,OPERATIONAL,ébar,ChIJ-_JYfG0Z2jERRqquqjPQ2_U,4.1,"['bar', 'point_of_interest', 'establishment']",24.0,57 Neil Road,Craig Road,AreaSearch,1.279538,103.842525,2.0,088893,088892,88893


In [ ]:
for postal_code in ['POSTALCODE_GOOGLE', 'POSTALCODE', 'POSTALCODE_COMBINED']:
    g_outlet_df_raw[postal_code] = g_outlet_df_raw[postal_code].str.zfill(6)

In [ ]:
g_outlet_df_raw['area_code'] = g_outlet_df_raw.apply(lambda x: str(x.POSTALCODE_COMBINED)[:area_digits], axis=1)

In [ ]:
g_outlet_df_raw.area_code.nunique()

1307

In [ ]:
g_outlet_df_raw.POSTALCODE.nunique()

5509

In [ ]:
# Get lat/lon bounds of scraped locations

# min_lat = min(min(g_outlet_df_raw['geometry.location.lat']), min(g_outlet_df_raw['geometry.viewport.northeast.lat']), min(g_outlet_df_raw['geometry.viewport.northeast.lat']))
# max_lat = max(max(g_outlet_df_raw['geometry.location.lat']), max(g_outlet_df_raw['geometry.viewport.northeast.lat']), max(g_outlet_df_raw['geometry.viewport.northeast.lat']))
# min_lon = min(min(g_outlet_df_raw['geometry.location.lng']), min(g_outlet_df_raw['geometry.viewport.northeast.lng']), min(g_outlet_df_raw['geometry.viewport.northeast.lng']))
# max_lon = max(max(g_outlet_df_raw['geometry.location.lng']), max(g_outlet_df_raw['geometry.viewport.northeast.lng']), max(g_outlet_df_raw['geometry.viewport.northeast.lng']))


# -- AND ((lat::float BETWEEN {{ min_lat }} AND {{ max_lat }}) OR lat is NULL)
# -- AND ((lon::float BETWEEN {{ min_lon }} AND {{ max_lon }}) OR lon is NULL)

## Comparing against ShopBack Outlets

In [ ]:
g_outlet_df = _deepnote_execute_sql('SELECT \n    business_status\n    , name\n    , rating\n    , user_ratings_total\n    , types\n    , price_level \n    , vicinity\n    , place_id\n    , location\n    , lat\n    , long\n    , POSTALCODE_COMBINED as POSTALCODE\n    , area_code\n    , type\n    -- , postalcode as postal_code\nFROM g_outlet_df_raw\nWHERE business_status = \'OPERATIONAL\'\nAND rating >= {{ min_rating }}\nORDER BY area_code, location', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
g_outlet_df

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,long,POSTALCODE,area_code,Type
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,103.850330,018961,0189,AreaSearch
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,103.850557,018961,0189,AreaSearch
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,103.850868,018961,0189,AreaSearch
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25632,OPERATIONAL,Ya Kun Kaya Toast (Waterway Point),3.9,621.0,"['cafe', 'store', 'restaurant', 'food', 'point...",1.0,"83 Punggol Central, B2-K12",ChIJEe3nr-QV2jERBZer6nIp4Dk,Waterway Point,1.406618,103.902438,828843,8288,AreaSearch
25633,OPERATIONAL,PappaRich Waterway Point,4.1,523.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"83 Punggol Central, #B1-17 Waterway Point",ChIJL_NVReMV2jERVupOfH4xR4Q,Waterway Point,1.406720,103.902454,828843,8288,AreaSearch
25634,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,103.902090,828868,8288,AreaSearch
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,103.902310,828843,8288,AreaSearch


In [ ]:
sb_outlet_df = _deepnote_execute_sql('WITH outlets_cleaned as (\n    SELECT DISTINCT \n        country\n        , brand_id\n        , brand_name \n        , outlet_id\n        , outlet_name\n        , outlet_type\n        \n        , is_public\n        , is_sb_pay_enabled\n        , is_sb_pay_later_enabled\n        , is_deleted\n        \n        , hoolah_store_uuid\n        , legacy_outlet_id_v2\n        , outlet_account_id\n        \n        , visa_payment_category\n        , mastercard_payment_category\n        , service_type\n        , sb_l1_category\n        , sb_l2_category\n        \n        ,COALESCE(salesforce_user_name) as brand_account_owner\n        ,COALESCE(country_internal_team) as country_internal_team\n        , outlet_last_excluded_refunded_sbpay_transaction_datetime_ltz::date AS last_outlet_sbpay_order_date\n        , location\n        , RIGHT(salesforce_postal_code, 6) as postal_code\n        , LEFT(postal_code, {{ area_digits }}) as area_code\n        , lat\n        , lon\n    FROM analytics_features.feature_outlets\n    WHERE country = {{ country }}\n    AND outlet_type = \'Physical\'\n    AND is_deleted != \'true\'\n    AND sb_l1_category = \'Food\'\n    AND ((lat is not NULL AND lon is not NULL) OR postal_code is NOT NULL) -- toggle if want to exclude NULL lat/lon values\n    AND not (lower(outlet_name) ilike \'%test%\'\n        or lower(outlet_name) ilike \'%do not use%\'\n        or lower(outlet_name) ilike \'%qa_prod%\')\n)\n\n, latest_opp AS ( -- missing bonus cashback rev and cash advance rev\n -- some product info may be missing: https://shopback.slack.com/archives/C7FRD51J4/p1706162689983779\n   select \n     case when country = \'Singapore\' then \'sg\'\n          when country = \'Malaysia\' then \'my\'\n     else \'au\' end as country,\n     analytics.ai_merchant_assets_sf.outlet_id,\n     sf_asset_status,\n     analytics.ai_merchant_assets_sf.product_name\n   from analytics.ai_merchant_assets_sf\n   left join analytics.ai_merchant_opportunity_line_item_sf\n          on analytics.ai_merchant_assets_sf.opportunity_product = analytics.ai_merchant_opportunity_line_item_sf.sf_opportunity_line_item_id\n   where analytics.ai_merchant_assets_sf.sf_asset_status in (\'Live\', \'Expired\')\n     and analytics.ai_merchant_assets_sf.country in (\'Singapore\')\n     --and analytics.ai_merchant_opportunity_line_item_sf.product_family \n      --in (\'Online Financial Services\',\'Offline Financial Services\',\'In-store\',\'In-store Vouchers\',\'Return Cashback\')\n     and coalesce(analytics.ai_merchant_assets_sf.product_name, analytics.ai_merchant_assets_sf.sf_asset_name, analytics.ai_merchant_opportunity_line_item_sf.sf_opportunity_line_item_description)\n       in (\'Offline PayLater\', \'Offline PayLater + Return Cashback (Apr 2023)\',\'Online PayLater\',\'Online Pay\',\'Offline Pay\',\'Offline Pay + Return Cashback (Apr 2023)\', \'Return Cashback\'\n       ,\'Card Linked Offer\',\'Card Linked Offer + Return Cashback (Apr 2023)\')\n     -- and analytics.ai_merchant_assets_sf.sf_opportunity_id = \'00684000002oUofAAE\'\n )\n\nSELECT\n  outlets_cleaned.*\n  , max(case when sf_asset_status = \'Live\' and is_public = 1 then \'Live\' else \'Expired\' end) as latest_sf_asset_status\n  , listagg(distinct product_name, \', \') as lifetime_assets\n  , listagg(distinct case when sf_asset_status in (\'Live\') and is_public = 1 then product_name else null end, \', \') as live_assets\nFROM outlets_cleaned\nINNER JOIN latest_opp\n       on outlets_cleaned.outlet_account_id = latest_opp.outlet_id\n       and outlets_cleaned.country = latest_opp.country\nGROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26\nORDER BY area_code', 'SQL_AA2E2A50_3B4E_4308_8E36_2E7182D0F779', audit_sql_comment='', sql_cache_mode='cache_disabled')
sb_outlet_df

,country,brand_id,brand_name,outlet_id,outlet_name,outlet_type,is_public,is_sb_pay_enabled,is_sb_pay_later_enabled,is_deleted,...,country_internal_team,last_outlet_sbpay_order_date,location,postal_code,area_code,lat,lon,latest_sf_asset_status,lifetime_assets,live_assets
0,sg,0114610,Jollibean,0000057698,Jollibean (All Outlets),Physical,False,False,False,false,...,SME,None,All Outlets,#,#,1.2779609,103.84446,Expired,Card Linked Offer,None
1,sg,0082156,Din Tai Fung,0000115626,Din Tai Fung (Marina Bay Link Mall),Physical,True,True,False,false,...,National 1,2024-06-20,Marina Bay Link Mall,018984,0189,1.2806453,103.8537135,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay"
2,sg,0187963,Mia tavola,0000185460,Mia tavola (Comic Con 2023),Physical,False,False,False,false,...,Others,2023-12-10,Comic Con 2023,018956,0189,1.2825554,103.8584627,Expired,"Offline Pay, Return Cashback",None
3,sg,0082156,Din Tai Fung,0000117267,Din Tai Fung (Marina Bay Sands),Physical,True,True,False,false,...,National 1,2024-06-20,Marina Bay Sands,018972,0189,1.2822001,103.8575756,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay"
4,sg,0252201,the moonbeam co. [Great Green Run],0000176890,the moonbeam co. [Great Green Run] (8 Marina G...,Physical,False,False,False,false,...,THLL,2023-09-24,8 Marina Gardens Drive,018951,0189,1.280681,103.8711213,Expired,"Return Cashback, Offline Pay",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7414,sg,0114161,Farisha's Kopi,0000052494,Farisha's Kopi (China Square),Physical,False,True,False,false,...,Others,None,China Square,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None
7415,sg,0114161,Farisha's Kopi,0000056615,Farisha's Kopi (Velocity),Physical,False,True,False,false,...,Others,None,Velocity,""":""#""}",None,1.4187298,104.2729082,Expired,Offline Pay + Return Cashback (Apr 2023),None
7416,sg,0114161,Farisha's Kopi,0000056990,Farisha's Kopi (Pasir Ris West Plaza),Physical,False,True,False,false,...,Others,None,Pasir Ris West Plaza,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None
7417,sg,0114161,Farisha's Kopi,0000059025,Farisha's Kopi (23 Serangoon Central),Physical,True,True,False,false,...,Others,None,23 Serangoon Central,""":""#""}",None,1.4187298,104.2729082,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card..."


In [ ]:
sb_outlet_df_csv = pd.read_csv('datasets/food outlets.csv')

In [ ]:
g_outlet_df.loc[g_outlet_df['area_code'].str.isnumeric() == False, 'area_code'] = 'misc'
sb_outlet_df.loc[sb_outlet_df['area_code'].str.isnumeric() == False, 'area_code'] = 'misc'
g_outlet_df.loc[g_outlet_df['area_code'] == '000', 'area_code'] = 'misc'
sb_outlet_df.loc[sb_outlet_df['area_code'] == '000', 'area_code'] = 'misc'

In [ ]:
g_outlet_df['index_key'] = g_outlet_df.index
sb_outlet_df['index_key'] = sb_outlet_df.index

In [ ]:
g_outlet_df.head()

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,long,POSTALCODE,area_code,Type,index_key
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch,0
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,103.850330,018961,0189,AreaSearch,1
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,103.850557,018961,0189,AreaSearch,2
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,103.850868,018961,0189,AreaSearch,3
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch,4


In [ ]:
g_outlet_df.area_code.nunique()

1307

In [ ]:
sb_outlet_df.area_code.nunique()

840

## Identifying uselss and mall words from Google and feature_outlet outlet names

In [ ]:
useless_names_google_api = pd.read_csv('datasets/useless_names_google_api.csv')
useless_names_feature_outlet = pd.read_csv('datasets/useless_names_feature_outlet.csv')

In [ ]:
def clean_names(df, col_name, suffix_list=['']):
    temp_df = df.copy()

    temp_df['clean_name'] = temp_df[col_name]
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: str(x))
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: x.lower()) # remove case sensitivity
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode()) # remove non ASCII characters (including chinese characters)
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: re.sub(r'[^\w\s]','', x)) # remove punctuation
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: basename(x)) # removes common corporate tags e.g Pte Ltd, LLC, Corp etc.
    # temp_df['clean_name'] = temp_df.apply(lambda x: ' '.join(re.sub(r'\b{}\b'.format(re.escape(word for word in x['location'].split(' '))), '', x).split()))

    for suffix in suffix_list:
        temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: ' '.join(re.sub(r'\b{}\b'.format(re.escape(suffix)), '', x).split())) # removes words in a pre-determined list
    
    temp_df['clean_name'] = temp_df['clean_name'].apply(lambda x: 'NA' if (x == '' or x is None or len(x) == 1) else x)
    # temp_df['clean_name'] = temp_df.apply(lambda x: f"{x.clean_name} {x.area_code}", axis=1)

    return temp_df

In [ ]:
def clean_names_additional(df, col_name, suffix_list=['']):
    temp_df = df.copy()
    new_cleaned_name = '{}_cleaned'.format(col_name)

    temp_df[new_cleaned_name] = temp_df[col_name]
    temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: str(x))
    temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: x.lower()) # remove case sensitivity
    temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: unicodedata.normalize('NFKD', x).encode('ASCII', 'ignore').decode()) # remove non ASCII characters (including chinese characters)
    temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: re.sub(r'[^\w\s]','', x)) # remove punctuation
    temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: basename(x)) # removes common corporate tags e.g Pte Ltd, LLC, Corp etc.

    # for suffix in suffix_list:
    #     temp_df[new_cleaned_name] = temp_df[new_cleaned_name].apply(lambda x: ' '.join(re.sub(r'\b{}\b'.format(re.escape(suffix)), '', x).split())) # removes words in a pre-determined list

    return temp_df.reset_index()

In [ ]:
def row_check(row, tracker_list):
    # Extend the tracker list with unique words from the row's location
    tracker_list.extend([word for word in row["Location_cleaned"].split(" ") if word not in tracker_list])

In [ ]:
list_useless_names_google_api = list(clean_names_additional(useless_names_google_api, 'keyword')['keyword_cleaned'])
list_useless_names_feature_outlet = list(clean_names_additional(useless_names_feature_outlet, 'keyword')['keyword_cleaned'])
clean_words = list(list_useless_names_google_api + list_useless_names_feature_outlet)

In [ ]:
# Data cleaning to improve false negatives / false positives
g_outlet_names = clean_names(g_outlet_df, 'name', clean_words)
# cleaning the location name 
g_outlet_names = clean_names_additional(g_outlet_names, 'Location')
sb_column_to_clean = 'outlet_name' if match_by == 'outlet' else 'brand_name'
sb_outlet_names = clean_names(sb_outlet_df, sb_column_to_clean, clean_words)

In [ ]:
area_code_list = list(g_outlet_names.area_code.unique())

In [ ]:
area_df = {}

for area_code in area_code_list:
    if area_code not in area_df:
        area_df[area_code] = {}
        
    area_df[area_code]['g_outlet_names'] = pd.DataFrame()
    area_df[area_code]['g_outlet_names'] = g_outlet_names[g_outlet_names['area_code'] == area_code]

    area_df[area_code]['clean_words'] = []
    for idx, row in g_outlet_names[g_outlet_names.area_code == area_code].iterrows():
        row_check(row, area_df[area_code]['clean_words'])
    
    area_df[area_code]['g_outlet_names'] = clean_names(area_df[area_code]['g_outlet_names'], 'clean_name', area_df[area_code]['clean_words'])

for area_code in sb_outlet_names.area_code.unique():
    if area_code not in area_df:
        area_df[area_code] = {}

    area_df[area_code]['sb_outlet_names'] = pd.DataFrame()
    area_df[area_code]['sb_outlet_names'] = sb_outlet_names[sb_outlet_names['area_code'] == area_code]

    if 'clean_words' in area_df[area_code]:
        area_df[area_code]['sb_outlet_names'] = clean_names(area_df[area_code]['sb_outlet_names'], 'clean_name', area_df[area_code]['clean_words'])

In [ ]:
# area_df['01890']['g_outlet_names']

In [ ]:
# area_df['01890']['sb_outlet_names']

In [ ]:
# area_df['01890']['clean_words']

In [ ]:
print(f"Running matches")
i = 0
for area_code in area_code_list:
    i+=1
    print(f"Area code: {area_code}")
    if ('sb_outlet_names' not in area_df[area_code]):
        print('SB outlet set empty')
        area_df[area_code]['sb_outlet_names'] = area_df['misc']['sb_outlet_names']
    g_outlet_names = area_df[area_code]['g_outlet_names']
    sb_outlet_names = area_df[area_code]['sb_outlet_names']
    adjusted_names = g_outlet_names
    test_names = sb_outlet_names
    # if i < 10:
    #     display(adjusted_names.head(5))
    #     display(test_names.head(5))

    '''
    Winning Variant:
    Run number: 36
    Parameters:
    ngrams: (2, 4)
    top_n: 50
    number_of_rows: 50
    '''
    '''
    Winning Variant
    Run number: 1146
    Parameters
    Distance metrics: ('iterative_sub_string', 'overlap', 'lig_3')
    '''

    ngrams = (2,4)
    top_n = 50
    number_of_rows = 50

    print(f"Parameters:")
    print(f"ngrams: {ngrams}")
    print(f"top_n: {top_n}")
    print(f"number_of_rows: {number_of_rows}")



    matcher = NameMatcher(  ngrams=ngrams,
                            top_n=top_n,
                            number_of_rows=number_of_rows,
                            number_of_matches=1,
                            lowercase=False,
                            punctuations=True,
                            remove_ascii=True,
                            legal_suffixes=False,
                            common_words=False,
                            preprocess_split=True,
                            verbose=False)

    # matcher.set_distance_metrics(['iterative_sub_string', 'pearson_ii', 'bag', 'fuzzy_wuzzy_partial_string', 'editex', 'discounted_levenshtein', 'SSK', 'fuzzy_wuzzy_token_sort'])
    # matcher.set_distance_metrics(['iterative_sub_string', 'pearson_ii', 'bag', 'fuzzy_wuzzy_partial_string', 'editex'])
    # matcher.set_distance_metrics(['discounted_levenshtein', 'SSK', 'fuzzy_wuzzy_token_sort'])
    matcher.set_distance_metrics(('iterative_sub_string', 'overlap', 'lig_3'))

    matcher.load_and_process_master_data('clean_name', test_names, transform=True)
    matches = matcher.match_names(to_be_matched=adjusted_names, column_matching='clean_name')
    # matches = pd.merge(g_outlet_names[g_outlet_names['area_code'] == area_code], matches, how='left', left_on='index_key', right_index=True)
    area_df[area_code]['matches'] = matches

top_n: 50
number_of_rows: 50
Area code: 6814
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6815
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6816
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6817
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6818
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6824
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6826
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6838
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6882
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6887
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6888
SB outlet set empty
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6892
SB outlet set empty
Parameters:
ngrams: (2,

In [ ]:
matches

,original_name,match_name,score,match_index
25597,kin kao mai thai,kin kao mai thai,100.000000,7339
25598,sum dim sum,sum dim sum,100.000000,7337
25599,pump station 1965 bistro,pump station 1965 bistro,100.000000,7342
25600,east container park,kin kao mai thai,13.445378,7339
25601,madya,kaizenki teppanyaki,9.259259,7340
25602,seoul good,big fish small fish,3.030303,7341
25603,na,drubinah,11.111111,7336
25604,kaizenki teppanyaki,kaizenki teppanyaki,100.000000,7340
25605,tenderbest makcik tuckshop east,kin kao mai thai,17.484278,7339
25606,fat po,cat fiddle,41.619418,7344


In [ ]:
matches_all = pd.DataFrame()
for area_code in area_df:
    if 'matches' in area_df[area_code]:
        matches_all = pd.concat([matches_all, area_df[area_code]['matches']])

matches_all = pd.concat([matches_all, matches])

In [ ]:
# import pandas as pd
# matches_all.to_csv('prelim_matches.csv')
# matches_all = pd.read_csv('prelim_matches.csv')
matches_all

,original_name,match_name,score,match_index
0,gochiso shokudo japanese,tsuta japanese soba noodles,31.136514,60
1,avorush,mia tavola,23.854240,2
2,subway,standing sushi,15.634921,51
3,paninoteca,nani bowl,35.879255,57
4,kpmg clubhouse,paradise classic,19.774563,45
...,...,...,...,...
25632,ya kun kaya toast,kin kao mai thai,30.525757,7339
25633,papparich,kaizenki teppanyaki,21.268385,7340
25634,ma la xiang guo,old chang kee,19.822754,7338
25635,sushi express,sum dim sum,18.412698,7337


In [ ]:
# Filter for scores above 70
matches_threshold = matches_all[matches_all['score'] >= 70]
matches_threshold

,original_name,match_name,score,match_index
9,liho tea,liho tea,100.000000,62
10,noodle,nuodle,80.193635,63
20,din tai fung,din tai fung,100.000000,3
35,good day,goodday,91.462963,48
39,nani bowl,nani bowl,100.000000,57
...,...,...,...,...
25597,kin kao mai thai,kin kao mai thai,100.000000,7339
25598,sum dim sum,sum dim sum,100.000000,7337
25599,pump station 1965 bistro,pump station 1965 bistro,100.000000,7342
25604,kaizenki teppanyaki,kaizenki teppanyaki,100.000000,7340


In [ ]:
not_matched_sb_outlet = sb_outlet_df[~sb_outlet_df['index_key'].isin(matches_threshold['match_index'])]
not_matched_sb_outlet

,country,brand_id,brand_name,outlet_id,outlet_name,outlet_type,is_public,is_sb_pay_enabled,is_sb_pay_later_enabled,is_deleted,...,last_outlet_sbpay_order_date,location,postal_code,area_code,lat,lon,latest_sf_asset_status,lifetime_assets,live_assets,index_key
0,sg,0114610,Jollibean,0000057698,Jollibean (All Outlets),Physical,False,False,False,false,...,None,All Outlets,#,misc,1.2779609,103.84446,Expired,Card Linked Offer,None,0
1,sg,0082156,Din Tai Fung,0000115626,Din Tai Fung (Marina Bay Link Mall),Physical,True,True,False,false,...,2024-06-20,Marina Bay Link Mall,018984,0189,1.2806453,103.8537135,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay",1
2,sg,0187963,Mia tavola,0000185460,Mia tavola (Comic Con 2023),Physical,False,False,False,false,...,2023-12-10,Comic Con 2023,018956,0189,1.2825554,103.8584627,Expired,"Offline Pay, Return Cashback",None,2
4,sg,0252201,the moonbeam co. [Great Green Run],0000176890,the moonbeam co. [Great Green Run] (8 Marina G...,Physical,False,False,False,false,...,2023-09-24,8 Marina Gardens Drive,018951,0189,1.280681,103.8711213,Expired,"Return Cashback, Offline Pay",None,4
5,sg,0091467,Boost Juice Bars,0000053355,Boost Juice Bars (Marina Bay Link Mall),Physical,True,True,False,false,...,2024-06-20,Marina Bay Link Mall,018984,0189,1.2804994,103.8536682,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7414,sg,0114161,Farisha's Kopi,0000052494,Farisha's Kopi (China Square),Physical,False,True,False,false,...,None,China Square,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,7414
7415,sg,0114161,Farisha's Kopi,0000056615,Farisha's Kopi (Velocity),Physical,False,True,False,false,...,None,Velocity,""":""#""}",None,1.4187298,104.2729082,Expired,Offline Pay + Return Cashback (Apr 2023),None,7415
7416,sg,0114161,Farisha's Kopi,0000056990,Farisha's Kopi (Pasir Ris West Plaza),Physical,False,True,False,false,...,None,Pasir Ris West Plaza,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,7416
7417,sg,0114161,Farisha's Kopi,0000059025,Farisha's Kopi (23 Serangoon Central),Physical,True,True,False,false,...,None,23 Serangoon Central,""":""#""}",None,1.4187298,104.2729082,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",7417


In [ ]:
complete_matched_data = pd.merge(g_outlet_df, matches_threshold, how='left', left_index=True, right_index=True)
complete_matched_data = pd.merge(complete_matched_data, sb_outlet_df, how='left', left_on='match_index', right_index=True)

In [ ]:
complete_matched_data.to_csv('complete_matched_data.csv')
complete_matched_data

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat_x,...,last_outlet_sbpay_order_date,location,postal_code,area_code_y,lat_y,lon,latest_sf_asset_status,lifetime_assets,live_assets,index_key_y
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25632,OPERATIONAL,Ya Kun Kaya Toast (Waterway Point),3.9,621.0,"['cafe', 'store', 'restaurant', 'food', 'point...",1.0,"83 Punggol Central, B2-K12",ChIJEe3nr-QV2jERBZer6nIp4Dk,Waterway Point,1.406618,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25633,OPERATIONAL,PappaRich Waterway Point,4.1,523.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"83 Punggol Central, #B1-17 Waterway Point",ChIJL_NVReMV2jERVupOfH4xR4Q,Waterway Point,1.406720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25634,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
complete_matched_data_no_threshold = pd.merge(g_outlet_df, matches_all, how='left', left_index=True, right_index=True)
complete_matched_data_no_threshold = pd.merge(complete_matched_data_no_threshold, sb_outlet_df, how='left', left_on='match_index', right_index=True)

In [ ]:
complete_matched_data_no_threshold.to_csv('complete_matched_data_no_threshold.csv')
complete_matched_data_no_threshold

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat_x,...,last_outlet_sbpay_order_date,location,postal_code,area_code_y,lat_y,lon,latest_sf_asset_status,lifetime_assets,live_assets,index_key_y
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,None,Gardens by the Bay,018953,0189,1.2821843,103.8657325,Expired,"Return Cashback, Offline Pay",None,60
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,2023-12-10,Comic Con 2023,018956,0189,1.2825554,103.8584627,Expired,"Offline Pay, Return Cashback",None,2
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,2024-06-20,Marina One,018935,0189,1.277901,103.8528438,Live,"Offline Pay, Return Cashback","Offline Pay, Return Cashback",51
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,2024-06-20,Marina One,018935,0189,1.2777581,103.852778,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),57
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,2024-06-20,Marina Bay Link Mall,018984,0189,1.2795633,103.85420169999998,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25634,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,2024-06-20,SPC Punggol,828839,8288,1.4003959,103.9091968,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),7338
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,2024-03-02,Punggol East,828826,8288,1.3935619,103.9177744,Expired,Offline Pay + Return Cashback (Apr 2023),None,7337
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,2024-03-02,Punggol East,828826,8288,1.3935619,103.9177744,Expired,Offline Pay + Return Cashback (Apr 2023),None,7337
25636,OPERATIONAL,MOS Burger,2.8,97.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #B1-K9 Waterway Point East...",ChIJSd_LA_gV2jERnZYdvlIIxhw,Waterway Point,1.406603,...,2024-06-20,Punggol East Container Park,828826,8288,1.3933497,103.9176661,Live,"Offline Pay, Return Cashback","Offline Pay, Return Cashback",7343


# Code for brand matching

In [ ]:
sb_brand_df = _deepnote_execute_sql('with outlets_cleaned as ( -- main clean outlets tale\n  select distinct\n      feature_outlets.country\n     ,feature_outlets.brand_id\n     ,feature_outlets.brand_name \n     ,feature_outlets.outlet_id\n     ,feature_outlets.outlet_name\n     ,feature_outlets.outlet_type\n     \n     ,feature_outlets.is_public\n     ,feature_outlets.is_sb_pay_enabled\n     ,feature_outlets.is_sb_pay_later_enabled\n     ,feature_outlets.is_deleted\n     \n     ,feature_outlets.hoolah_store_uuid\n     ,feature_outlets.legacy_outlet_id_v2\n     ,feature_outlets.outlet_account_id\n     \n     ,feature_outlets.sb_l1_category\n     ,feature_outlets.sb_l2_category\n     \n     ,COALESCE(feature_outlets.salesforce_user_name) as brand_account_owner\n     ,COALESCE(feature_outlets.country_internal_team) as country_internal_team\n     ,outlet_last_excluded_refunded_sbpay_transaction_datetime_ltz::date AS last_outlet_sbpay_order_date\n\n  from analytics_features.feature_outlets\n  where analytics_features.feature_outlets.country = {{ country }}\n  --  and (analytics.feature_outlets.is_sb_pay_enabled)\n --   and analytics.feature_outlets.is_public = 1\n   -- and analytics.feature_outlets.is_deleted = 0\n    and not (lower(feature_outlets.outlet_name) ilike \'%test%\'\n       or lower(feature_outlets.outlet_name) ilike \'%do not use%\'\n       or lower(feature_outlets.outlet_name) ilike \'%qa_prod%\')\n ),\n \n latest_opp AS ( -- missing bonus cashback rev and cash advance rev\n -- some product info may be missing: https://shopback.slack.com/archives/C7FRD51J4/p1706162689983779\n   select \n     case when country = \'Singapore\' then \'sg\'\n          when country = \'Malaysia\' then \'my\'\n     else \'au\' end as country,\n     analytics.ai_merchant_assets_sf.outlet_id,\n     sf_asset_status,\n     analytics.ai_merchant_assets_sf.product_name\n   from analytics.ai_merchant_assets_sf\n   left join analytics.ai_merchant_opportunity_line_item_sf\n          on analytics.ai_merchant_assets_sf.opportunity_product = analytics.ai_merchant_opportunity_line_item_sf.sf_opportunity_line_item_id\n   where analytics.ai_merchant_assets_sf.sf_asset_status in (\'Live\', \'Expired\')\n     and analytics.ai_merchant_assets_sf.country in (\'Singapore\')\n     --and analytics.ai_merchant_opportunity_line_item_sf.product_family \n      --in (\'Online Financial Services\',\'Offline Financial Services\',\'In-store\',\'In-store Vouchers\',\'Return Cashback\')\n     and coalesce(analytics.ai_merchant_assets_sf.product_name, analytics.ai_merchant_assets_sf.sf_asset_name, analytics.ai_merchant_opportunity_line_item_sf.sf_opportunity_line_item_description)\n       in (\'Offline PayLater\', \'Offline PayLater + Return Cashback (Apr 2023)\',\'Online PayLater\',\'Online Pay\',\'Offline Pay\',\'Offline Pay + Return Cashback (Apr 2023)\', \'Return Cashback\'\n       ,\'Card Linked Offer\',\'Card Linked Offer + Return Cashback (Apr 2023)\')\n     -- and analytics.ai_merchant_assets_sf.sf_opportunity_id = \'00684000002oUofAAE\'\n ),\n\nbrands_sf as (\nselect distinct\n  brand_id,\n  sf_account_name,\n  sf_account_owner\nfrom analytics.ai_merchant_accounts_sf\nwhere country = {{ country }}\n  and sf_account_record_type_text = \'Brand\'\n  and is_deleted = \'false\'\n  and (sb_l1_category = \'Food\' or sb_l1_category is null)\n),\n\nbrands_outlets as (\nSELECT \n  outlets_cleaned.brand_id,\n  outlets_cleaned.brand_name, \n  outlets_cleaned.brand_account_owner,\n  max(case when sf_asset_status = \'Live\' and is_public = 1 then \'Live\' else \'Expired\' end) as latest_sf_asset_status,\n  listagg(distinct product_name, \', \') as lifetime_assets,\n  listagg(distinct case when sf_asset_status in (\'Live\') and is_public = 1 then product_name else null end, \', \') as live_assets\nFROM outlets_cleaned\nINNER JOIN latest_opp\n       on outlets_cleaned.outlet_account_id = latest_opp.outlet_id\n       and outlets_cleaned.country = latest_opp.country\ngroup by 1,2,3\n)\n\nselect\n  brands_sf.*,\n  coalesce(brands_outlets.latest_sf_asset_status, \'Created\') as latest_sf_asset_status,\n  brands_outlets.lifetime_assets,\n  brands_outlets.live_assets\nfrom brands_sf\nleft join brands_outlets \n       on brands_sf.brand_id = brands_outlets.brand_id\norder by 4 desc, 2 asc', 'SQL_AA2E2A50_3B4E_4308_8E36_2E7182D0F779', audit_sql_comment='', sql_cache_mode='cache_disabled')
sb_brand_df

,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback..."
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023)
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023)
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu..."
...,...,...,...,...,...,...
21469,0184337,香港車仔麵之家 Little Cart Noodle House,cassandra pek,Created,None,None
21470,0253802,鸿兴潮州粿,cassandra pek,Created,None,None
21471,0253806,鸿兴潮州鱼圆面,cassandra pek,Created,None,None
21472,0096813,麻麻辣 Ma Ma La,Elianie Murni,Created,None,None


In [ ]:
sb_brand_df

,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback..."
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023)
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023)
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu..."
...,...,...,...,...,...,...
21469,0184337,香港車仔麵之家 Little Cart Noodle House,cassandra pek,Created,None,None
21470,0253802,鸿兴潮州粿,cassandra pek,Created,None,None
21471,0253806,鸿兴潮州鱼圆面,cassandra pek,Created,None,None
21472,0096813,麻麻辣 Ma Ma La,Elianie Murni,Created,None,None


In [ ]:
sb_brand_names = clean_names(sb_brand_df, 'sf_account_name', clean_words)

In [ ]:
sb_brand_names

,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets,clean_name
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback...",0125 drinks
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan economic noodle
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan yu piao zhu du tang
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer,working title burger
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu...",001
...,...,...,...,...,...,...,...
21469,0184337,香港車仔麵之家 Little Cart Noodle House,cassandra pek,Created,None,None,little cart noodle
21470,0253802,鸿兴潮州粿,cassandra pek,Created,None,None,NA
21471,0253806,鸿兴潮州鱼圆面,cassandra pek,Created,None,None,NA
21472,0096813,麻麻辣 Ma Ma La,Elianie Murni,Created,None,None,ma ma la


In [ ]:
print(f"Running matches")
i = 0
for area_code in area_code_list:
    i+=1
    print(f"Area code: {area_code}")
    g_outlet_names = area_df[area_code]['g_outlet_names']
    adjusted_names = g_outlet_names
    test_names = sb_brand_names
    if i < 10:
        display(adjusted_names.head(5))
        display(test_names.head(5))

    '''
    Winning Variant:
    Run number: 36
    Parameters:
    ngrams: (2, 4)
    top_n: 50
    number_of_rows: 50
    '''
    '''
    Winning Variant
    Run number: 1146
    Parameters
    Distance metrics: ('iterative_sub_string', 'overlap', 'lig_3')
    '''

    ngrams = (2,4)
    top_n = 50
    number_of_rows = 50

    print(f"Parameters:")
    print(f"ngrams: {ngrams}")
    print(f"top_n: {top_n}")
    print(f"number_of_rows: {number_of_rows}")



    matcher = NameMatcher(  ngrams=ngrams,
                            top_n=top_n,
                            number_of_rows=number_of_rows,
                            number_of_matches=1,
                            lowercase=False,
                            punctuations=True,
                            remove_ascii=True,
                            legal_suffixes=False,
                            common_words=False,
                            preprocess_split=True,
                            verbose=False)

    # matcher.set_distance_metrics(['iterative_sub_string', 'pearson_ii', 'bag', 'fuzzy_wuzzy_partial_string', 'editex', 'discounted_levenshtein', 'SSK', 'fuzzy_wuzzy_token_sort'])
    # matcher.set_distance_metrics(['iterative_sub_string', 'pearson_ii', 'bag', 'fuzzy_wuzzy_partial_string', 'editex'])
    # matcher.set_distance_metrics(['discounted_levenshtein', 'SSK', 'fuzzy_wuzzy_token_sort'])
    matcher.set_distance_metrics(('iterative_sub_string', 'overlap', 'lig_3'))

    matcher.load_and_process_master_data('clean_name', test_names, transform=True)
    matches = matcher.match_names(to_be_matched=adjusted_names, column_matching='clean_name')
    # matches = pd.merge(g_outlet_names[g_outlet_names['area_code'] == area_code], matches, how='left', left_on='index_key', right_index=True)
    area_df[area_code]['matches'] = matches

Running matches
Area code: 0189


,index,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,long,POSTALCODE,area_code,Type,index_key,clean_name,Location_cleaned
0,0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch,0,gochiso shokudo japanese,afro asia
1,1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,103.850330,018961,0189,AreaSearch,1,avorush,afro asia
2,2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,103.850557,018961,0189,AreaSearch,2,subway,afro asia
3,3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,103.850868,018961,0189,AreaSearch,3,paninoteca,afro asia
4,4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,103.850879,018961,0189,AreaSearch,4,kpmg clubhouse,afro asia


,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets,clean_name
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback...",0125 drinks
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan economic noodle
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan yu piao zhu du tang
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer,working title burger
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu...",001


Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 0389


,index,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,long,POSTALCODE,area_code,Type,index_key,clean_name,Location_cleaned
240,240,OPERATIONAL,Fort Sanctuary,4.8,23.0,"['cafe', 'point_of_interest', 'food', 'establi...",NaN,"8 Raffles Avenue, #02-04 Esplanade Mall",ChIJLz4DGkUZ2jERq3BolsWBFl4,CityLink Mall,1.289548,103.855697,038981,0389,AreaSearch,240,fort sanctuary,citylink mall
241,241,OPERATIONAL,Red House Seafood at Esplanade,4.4,172.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"8 Raffles Avenue, #01-14 / 16 Esplanade",ChIJSW7NnhAZ2jERj8p3W_nnhGE,CityLink Mall,1.289659,103.855763,038981,0389,AreaSearch,241,red seafood at,citylink mall
242,242,OPERATIONAL,Chun Fun How Singapore 春芳號,4.7,170.0,"['cafe', 'point_of_interest', 'food', 'store',...",NaN,"8 Raffles Avenue, #02-13 The Esplanade Mall",ChIJ5x33EC4Z2jERN6WDLnGju8Y,CityLink Mall,1.289659,103.855763,038981,0389,AreaSearch,242,chun fun how,citylink mall
243,243,OPERATIONAL,ARKTOS GELATO,4.9,12.0,"['cafe', 'point_of_interest', 'food', 'store',...",NaN,"8 Raffles Avenue, Esplanade Mall #01-13 Annexe...",ChIJd1awt3sZ2jERavEalXhIZ1w,CityLink Mall,1.289659,103.855763,038981,0389,AreaSearch,243,arktos gelato,citylink mall
244,244,OPERATIONAL,Uncle Fong Hotpot Restaurant (Suntec City) 方叔叔...,4.9,284.0,"['restaurant', 'point_of_interest', 'food', 'e...",NaN,"3 Temasek Boulevard, Suntec City #02-391, Atri...",ChIJ8em5PG8Z2jER76GdFZZL3Oo,Esplanade Xchange,1.293631,103.857366,038985,0389,AreaSearch,244,uncle fong hotpot,esplanade xchange


...

,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets,clean_name
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback...",0125 drinks
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan economic noodle
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan yu piao zhu du tang
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer,working title burger
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu...",001


Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 0485


,index,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,long,POSTALCODE,area_code,Type,index_key,clean_name,Location_cleaned
614,614,OPERATIONAL,LiXin Teochew Fishball Noodles - Lau Pa Sat,3.5,17.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,18 Raffles Quay,ChIJ5Y6l4Q0Z2jER05rmECQ81wk,Afro Asia,1.280380,103.850210,048582,0485,AreaSearch,614,lixin teochew fishball noodles lau pa sat,afro asia
615,615,OPERATIONAL,Ipoh Hainanese Chicken Rice Halal,3.7,11.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"18 Raffles Quay, Stall 14 Lau Pa Sat",ChIJC3xj6QcZ2jERtX4PzPMyGXQ,Afro Asia,1.280371,103.850320,048582,0485,AreaSearch,615,ipoh hainanese chicken rice,afro asia
616,616,OPERATIONAL,Qiu Lian Ban Mian,2.4,49.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"18 Raffles Quay, Singapore",ChIJ5Y6l4Q0Z2jERk5zvoDtDsfs,Afro Asia,1.280630,103.850251,048582,0485,AreaSearch,616,qiu lian ban mian,afro asia
617,617,OPERATIONAL,Lobby Lounge at The Westin Singapore,4.2,102.0,"['night_club', 'bar', 'point_of_interest', 'es...",NaN,"12 Marina View, Asia Square Tower 2 Level 32, ...",ChIJpd8riREZ2jERhXWkEj5819c,Afro Asia,1.278278,103.850772,048580,0485,AreaSearch,617,lobby lounge at westin,afro asia
618,618,OPERATIONAL,Lau Pa Sat,4.3,27548.0,"['tourist_attraction', 'restaurant', 'food', '...",NaN,18 Raffles Quay,ChIJ5Y6l4Q0Z2jERYL0KDIjT6v0,Afro Asia,1.280512,103.850381,048582,0485,AreaSearch,618,lau pa sat,afro asia


,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets,clean_name
0,0125120,#01-25 Drinks,Automation Bot,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay + Return Cashback...",0125 drinks
1,0233568,(Le Ning Xuan) Economic Noodle,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan economic noodle
2,0233260,(Le Ning Xuan) Yu Piao Zhu Du Tang,Automation Bot,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),le ning xuan yu piao zhu du tang
3,0089285,(working title) Burger Bar,Product Payments,Live,Card Linked Offer,Card Linked Offer,working title burger
4,0078513,001 Café,Hafiz Saleh,Live,"Offline Pay + Return Cashback (Apr 2023), Retu...","Offline Pay + Return Cashback (Apr 2023), Retu...",001


Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6780
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6781
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6782
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6790
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6796
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6798
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6799
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6800
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6801
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6802
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6803
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6804
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area code: 6805
Parameters:
ngrams: (2, 4)
top_n: 50
number_of_rows: 50
Area cod

In [ ]:
matches_all = pd.DataFrame()
for area_code in area_df:
    if 'matches' in area_df[area_code]:
        matches_all = pd.concat([matches_all, area_df[area_code]['matches']])

matches_all = pd.concat([matches_all, matches])

In [ ]:
# import pandas as pd
# matches_all.to_csv('prelim_matches.csv')
# matches_all = pd.read_csv('prelim_matches.csv')
matches_all

,original_name,match_name,score,match_index
0,gochiso shokudo japanese,gochiso shokudo,87.916667,9304
1,avorush,avorush,100.000000,5088
2,subway,subway,100.000000,1887
3,paninoteca,paninoteca,100.000000,14776
4,kpmg clubhouse,clubhouse,59.710145,7018
...,...,...,...,...
25632,ya kun kaya toast,ya kun kaya toast,100.000000,20355
25633,papparich,papparich,100.000000,14799
25634,ma la xiang guo,mala xiangguo,83.138115,12871
25635,sushi express,sushi express,100.000000,1908


In [ ]:
# Filter for scores above 70
matches_threshold = matches_all[matches_all['score'] >= 70]
matches_threshold

,original_name,match_name,score,match_index
0,gochiso shokudo japanese,gochiso shokudo,87.916667,9304
1,avorush,avorush,100.000000,5088
2,subway,subway,100.000000,1887
3,paninoteca,paninoteca,100.000000,14776
6,hakka yong tau foo,khaki yong tau foo,87.603955,1180
...,...,...,...,...
25632,ya kun kaya toast,ya kun kaya toast,100.000000,20355
25633,papparich,papparich,100.000000,14799
25634,ma la xiang guo,mala xiangguo,83.138115,12871
25635,sushi express,sushi express,100.000000,1908


In [ ]:
complete_matched_brands = pd.merge(g_outlet_df, matches_threshold, how='left', left_index=True, right_index=True)
complete_matched_brands = pd.merge(complete_matched_brands, sb_brand_df, how='left', left_on='match_index', right_index=True)

In [ ]:
# dedup here with place_id
complete_matched_brands = complete_matched_brands.drop_duplicates(
        subset='place_id', keep='first')
complete_matched_brands.to_csv('complete_matched_brands.csv')
complete_matched_brands

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,...,original_name,match_name,score,match_index,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,gochiso shokudo japanese,gochiso shokudo,87.916667,9304.0,0125296,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,avorush,avorush,100.000000,5088.0,0085973,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,subway,subway,100.000000,1887.0,0093911,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,paninoteca,paninoteca,100.000000,14776.0,0093519,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25632,OPERATIONAL,Ya Kun Kaya Toast (Waterway Point),3.9,621.0,"['cafe', 'store', 'restaurant', 'food', 'point...",1.0,"83 Punggol Central, B2-K12",ChIJEe3nr-QV2jERBZer6nIp4Dk,Waterway Point,1.406618,...,ya kun kaya toast,ya kun kaya toast,100.000000,20355.0,0114937,Ya Kun Kaya Toast,Yuping Goh,Created,None,None
25633,OPERATIONAL,PappaRich Waterway Point,4.1,523.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"83 Punggol Central, #B1-17 Waterway Point",ChIJL_NVReMV2jERVupOfH4xR4Q,Waterway Point,1.406720,...,papparich,papparich,100.000000,14799.0,0076481,PappaRich Group,Yolanda Tan,Created,None,None
25634,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,ma la xiang guo,mala xiangguo,83.138115,12871.0,0098861,Mala XiangGuo,Cassia Tok,Created,None,None
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,sushi express,sushi express,100.000000,1908.0,0101346,Sushi Express,Yuping Goh,Live,Card Linked Offer,Card Linked Offer


In [ ]:
complete_matched_brands_no_threshold = pd.merge(g_outlet_df, matches_all, how='left', left_index=True, right_index=True)
complete_matched_brands_no_threshold = pd.merge(complete_matched_brands_no_threshold, sb_brand_df, how='left', left_on='match_index', right_index=True)

In [ ]:
# dedup here with place_id
complete_matched_brands_no_threshold = complete_matched_brands_no_threshold.drop_duplicates(
        subset='place_id', keep='first')
complete_matched_brands_no_threshold.to_csv('complete_matched_brands_no_threshold.csv')
complete_matched_brands_no_threshold

,business_status,name,rating,user_ratings_total,types,price_level,vicinity,place_id,Location,lat,...,original_name,match_name,score,match_index,brand_id,sf_account_name,sf_account_owner,latest_sf_asset_status,lifetime_assets,live_assets
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,gochiso shokudo japanese,gochiso shokudo,87.916667,9304,0125296,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,avorush,avorush,100.000000,5088,0085973,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,subway,subway,100.000000,1887,0093911,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,paninoteca,paninoteca,100.000000,14776,0093519,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,kpmg clubhouse,clubhouse,59.710145,7018,0119203,Clubhouse SG,Alavie Lee,Created,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25632,OPERATIONAL,Ya Kun Kaya Toast (Waterway Point),3.9,621.0,"['cafe', 'store', 'restaurant', 'food', 'point...",1.0,"83 Punggol Central, B2-K12",ChIJEe3nr-QV2jERBZer6nIp4Dk,Waterway Point,1.406618,...,ya kun kaya toast,ya kun kaya toast,100.000000,20355,0114937,Ya Kun Kaya Toast,Yuping Goh,Created,None,None
25633,OPERATIONAL,PappaRich Waterway Point,4.1,523.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"83 Punggol Central, #B1-17 Waterway Point",ChIJL_NVReMV2jERVupOfH4xR4Q,Waterway Point,1.406720,...,papparich,papparich,100.000000,14799,0076481,PappaRich Group,Yolanda Tan,Created,None,None
25634,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,ma la xiang guo,mala xiangguo,83.138115,12871,0098861,Mala XiangGuo,Cassia Tok,Created,None,None
25635,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,sushi express,sushi express,100.000000,1908,0101346,Sushi Express,Yuping Goh,Live,Card Linked Offer,Card Linked Offer


In [ ]:
with_threshold = pd.merge(complete_matched_data, complete_matched_brands, how='left', left_on='index_key_x', right_on='index_key')
no_threshold = pd.merge(complete_matched_data_no_threshold, complete_matched_brands_no_threshold, how='left', left_on='index_key_x', right_on='index_key')

In [ ]:
final_with_threshold_df = _deepnote_execute_sql('-- final_matched_data_brands_df\n-- We will select the needed columns here\nSELECT \n    -- Google Info\n    business_status_x AS business_status\n    , name_x AS outlet_name\n    , rating_x AS outlet_rating\n    , user_ratings_total_x AS outlet_ratings\n    , types_x AS type\n    , price_level_x AS price\n    , vicinity_x AS street\n    , place_id_x AS place_id\n    , Location_x AS nearby_mall_busstop\n    , lat_x AS lat\n    , long_x AS long\n    , POSTALCODE_x as POSTALCODE\n    , area_code\n\n    -- Match by google outlet name <> brand_name info area code level score\n    , score_x AS score\n    -- Match by google outlet name <> brand_name info area code level \n    , outlet_name AS outlet\n    , brand_account_owner AS owner\n    , latest_sf_asset_status_x AS outlet_live\n    , lifetime_assets_x AS lifetime_assets\n    , live_assets_x AS live_assets\n\n    -- Match google outlet name <> brand_name score\n    , score_y AS score_sf_brand\n    -- Match google outlet name <> brand_name\n    , sf_account_name AS brand\n    , sf_account_owner AS owner_sf_brand\n    , latest_sf_asset_status_y AS brand_live\n    , lifetime_assets_y AS lifetime_assets_sf_brand\n    , live_assets_y AS live_assets_sf_brand\n    -- , postalcode as postal_code\nFROM with_threshold', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
final_with_threshold_df

,business_status,outlet_name,outlet_rating,outlet_ratings,type,price,street,place_id,nearby_mall_busstop,lat,...,owner,outlet_live,lifetime_assets,live_assets,score_sf_brand,brand,owner_sf_brand,brand_live,lifetime_assets_sf_brand,live_assets_sf_brand
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,None,None,None,None,87.916667,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,None,None,None,None,100.000000,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,None,None,None,None,100.000000,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,None,None,None,None,100.000000,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,None,None,None,None,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25637,OPERATIONAL,Ya Kun Kaya Toast (Waterway Point),3.9,621.0,"['cafe', 'store', 'restaurant', 'food', 'point...",1.0,"83 Punggol Central, B2-K12",ChIJEe3nr-QV2jERBZer6nIp4Dk,Waterway Point,1.406618,...,None,None,None,None,100.000000,Ya Kun Kaya Toast,Yuping Goh,Created,None,None
25638,OPERATIONAL,PappaRich Waterway Point,4.1,523.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"83 Punggol Central, #B1-17 Waterway Point",ChIJL_NVReMV2jERVupOfH4xR4Q,Waterway Point,1.406720,...,None,None,None,None,100.000000,PappaRich Group,Yolanda Tan,Created,None,None
25639,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,None,None,None,None,83.138115,Mala XiangGuo,Cassia Tok,Created,None,None
25640,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,None,None,None,None,100.000000,Sushi Express,Yuping Goh,Live,Card Linked Offer,Card Linked Offer


In [ ]:
final_no_threshold_df = _deepnote_execute_sql('-- We will select the needed columns here\nSELECT \n    -- Google Info\n    business_status_x AS business_status\n    , name_x AS outlet_name\n    , rating_x AS outlet_rating\n    , user_ratings_total_x AS outlet_ratings\n    , types_x AS type\n    , price_level_x AS price\n    , vicinity_x AS street\n    , place_id_x AS place_id\n    , Location_x AS nearby_mall_busstop\n    , lat_x AS lat\n    , long_x AS long\n    , POSTALCODE_x as POSTALCODE\n    , area_code\n\n    -- Match by google outlet name <> brand_name info area code level score\n    , score_x AS score\n    -- Match by google outlet name <> brand_name info area code level \n    , outlet_name AS outlet\n    , brand_account_owner AS owner\n    , latest_sf_asset_status_x AS outlet_live\n    , lifetime_assets_x AS lifetime_assets\n    , live_assets_x AS live_assets\n\n    -- Match google outlet name <> brand_name score\n    , score_y AS score_sf_brand\n    -- Match google outlet name <> brand_name\n    , sf_account_name AS brand\n    , sf_account_owner AS owner_sf_brand\n    , latest_sf_asset_status_y AS brand_live\n    , lifetime_assets_y AS lifetime_assets_sf_brand\n    , live_assets_y AS live_assets_sf_brand\n    -- , postalcode as postal_code\nFROM no_threshold', 'SQL_DEEPNOTE_DATAFRAME_SQL', audit_sql_comment='', sql_cache_mode='cache_disabled')
final_no_threshold_df

,business_status,outlet_name,outlet_rating,outlet_ratings,type,price,street,place_id,nearby_mall_busstop,lat,...,owner,outlet_live,lifetime_assets,live_assets,score_sf_brand,brand,owner_sf_brand,brand_live,lifetime_assets_sf_brand,live_assets_sf_brand
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,Izzy Teh,Expired,"Return Cashback, Offline Pay",None,87.916667,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,Jonathan Tan,Expired,"Offline Pay, Return Cashback",None,100.000000,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,cassandra pek,Live,"Offline Pay, Return Cashback","Offline Pay, Return Cashback",100.000000,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,Hafiz Saleh,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),100.000000,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,Yuping Goh,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",59.710145,Clubhouse SG,Alavie Lee,Created,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25672,OPERATIONAL,Ma La Xiang Guo,2.0,23.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,83 Punggol Central,ChIJX9X_WeMV2jERatx-24MKxps,Waterway Point,1.405785,...,Jones Lee,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),83.138115,Mala XiangGuo,Cassia Tok,Created,None,None
25673,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,Hafiz Saleh,Expired,Offline Pay + Return Cashback (Apr 2023),None,100.000000,Sushi Express,Yuping Goh,Live,Card Linked Offer,Card Linked Offer
25674,OPERATIONAL,Sushi Express - Waterway Point,3.6,643.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #01-25 Waterway Point",ChIJq5FXpeQV2jERvqbXmDtJPAc,Waterway Point,1.406690,...,Hafiz Saleh,Expired,Offline Pay + Return Cashback (Apr 2023),None,100.000000,Sushi Express,Yuping Goh,Live,Card Linked Offer,Card Linked Offer
25675,OPERATIONAL,MOS Burger,2.8,97.0,"['restaurant', 'food', 'point_of_interest', 'e...",1.0,"83 Punggol Central, #B1-K9 Waterway Point East...",ChIJSd_LA_gV2jERnZYdvlIIxhw,Waterway Point,1.406603,...,Automation Bot,Live,"Offline Pay, Return Cashback","Offline Pay, Return Cashback",100.000000,MOS Burger,Yuping Goh,Created,None,None


In [ ]:
not_matched_sb_outlet

,country,brand_id,brand_name,outlet_id,outlet_name,outlet_type,is_public,is_sb_pay_enabled,is_sb_pay_later_enabled,is_deleted,...,last_outlet_sbpay_order_date,location,postal_code,area_code,lat,lon,latest_sf_asset_status,lifetime_assets,live_assets,index_key
0,sg,0114610,Jollibean,0000057698,Jollibean (All Outlets),Physical,False,False,False,false,...,None,All Outlets,#,misc,1.2779609,103.84446,Expired,Card Linked Offer,None,0
1,sg,0082156,Din Tai Fung,0000115626,Din Tai Fung (Marina Bay Link Mall),Physical,True,True,False,false,...,2024-06-20,Marina Bay Link Mall,018984,0189,1.2806453,103.8537135,Live,"Return Cashback, Offline Pay + Return Cashback...","Return Cashback, Offline Pay",1
2,sg,0187963,Mia tavola,0000185460,Mia tavola (Comic Con 2023),Physical,False,False,False,false,...,2023-12-10,Comic Con 2023,018956,0189,1.2825554,103.8584627,Expired,"Offline Pay, Return Cashback",None,2
4,sg,0252201,the moonbeam co. [Great Green Run],0000176890,the moonbeam co. [Great Green Run] (8 Marina G...,Physical,False,False,False,false,...,2023-09-24,8 Marina Gardens Drive,018951,0189,1.280681,103.8711213,Expired,"Return Cashback, Offline Pay",None,4
5,sg,0091467,Boost Juice Bars,0000053355,Boost Juice Bars (Marina Bay Link Mall),Physical,True,True,False,false,...,2024-06-20,Marina Bay Link Mall,018984,0189,1.2804994,103.8536682,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7414,sg,0114161,Farisha's Kopi,0000052494,Farisha's Kopi (China Square),Physical,False,True,False,false,...,None,China Square,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,7414
7415,sg,0114161,Farisha's Kopi,0000056615,Farisha's Kopi (Velocity),Physical,False,True,False,false,...,None,Velocity,""":""#""}",None,1.4187298,104.2729082,Expired,Offline Pay + Return Cashback (Apr 2023),None,7415
7416,sg,0114161,Farisha's Kopi,0000056990,Farisha's Kopi (Pasir Ris West Plaza),Physical,False,True,False,false,...,None,Pasir Ris West Plaza,""":""#""}",None,1.4187298,104.2729082,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,7416
7417,sg,0114161,Farisha's Kopi,0000059025,Farisha's Kopi (23 Serangoon Central),Physical,True,True,False,false,...,None,23 Serangoon Central,""":""#""}",None,1.4187298,104.2729082,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",7417


In [ ]:
final_with_threshold_df_dedup = final_with_threshold_df.drop_duplicates(
            subset='place_id', keep='first')

final_no_threshold_df_dedup = final_no_threshold_df.drop_duplicates(
            subset='place_id', keep='first')

# add the not matched outlets to both final_matched_data_brands_df_dedup and final_matched_brands_df_dedup
not_matched_sb_outlet_to_use = not_matched_sb_outlet[[
'outlet_name',
'location',
'lat',
'lon',
'postal_code',
'area_code',
'brand_account_owner',
'latest_sf_asset_status', 
'lifetime_assets',
'live_assets']]

not_matched_sb_outlet_renamed = not_matched_sb_outlet_to_use.rename(columns={
    'location': 'street',
    'lat': 'lat',
    'lon': 'long',
    'postal_code': 'POSTALCODE',
    'area_code': 'area_code',
    'outlet_name' : 'outlet',
    'brand_account_owner': 'owner',
    'latest_sf_asset_status':'outlet_live',
})

for col in final_with_threshold_df_dedup.columns:
    if col not in not_matched_sb_outlet_renamed.columns:
        not_matched_sb_outlet_renamed[col] = ''

for col in final_no_threshold_df_dedup.columns:
    if col not in not_matched_sb_outlet_renamed.columns:
        not_matched_sb_outlet_renamed[col] = ''

# We need to be able to add these columns into 
# Lat <> lat
# Lon <> lon
# Postal Code <> postal_code 
# Area Code <> area_code
# Street <> location

final_with_threshold_df_combined_complete = pd.concat([final_with_threshold_df_dedup, not_matched_sb_outlet_renamed], ignore_index=True)
final_no_threshold_df_combined_complete = pd.concat([final_no_threshold_df_dedup, not_matched_sb_outlet_renamed], ignore_index=True)


final_with_threshold_df_combined.to_csv('final_with_threshold_df_combined_complete.csv')
final_no_threshold_df_combined.to_csv('final_no_threshold_df_combined_complete.csv')

In [ ]:
final_with_threshold_df_combined_complete

,business_status,outlet_name,outlet_rating,outlet_ratings,type,price,street,place_id,nearby_mall_busstop,lat,...,owner,outlet_live,lifetime_assets,live_assets,score_sf_brand,brand,owner_sf_brand,brand_live,lifetime_assets_sf_brand,live_assets_sf_brand
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,None,None,None,None,87.916667,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,None,None,None,None,100.0,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,None,None,None,None,100.0,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,None,None,None,None,100.0,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,None,None,None,None,NaN,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30287,,,,,,,China Square,,,1.4187298,...,Automation Bot,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,,,,,,
30288,,,,,,,Velocity,,,1.4187298,...,Automation Bot,Expired,Offline Pay + Return Cashback (Apr 2023),None,,,,,,
30289,,,,,,,Pasir Ris West Plaza,,,1.4187298,...,Automation Bot,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,,,,,,
30290,,,,,,,23 Serangoon Central,,,1.4187298,...,Automation Bot,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",,,,,,


In [ ]:
final_no_threshold_df_combined_complete

,business_status,outlet_name,outlet_rating,outlet_ratings,type,price,street,place_id,nearby_mall_busstop,lat,...,owner,outlet_live,lifetime_assets,live_assets,score_sf_brand,brand,owner_sf_brand,brand_live,lifetime_assets_sf_brand,live_assets_sf_brand
0,OPERATIONAL,Gochi-So Shokudo Japanese Restaurant ごち惣食堂 @ A...,4.1,29.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"Asia Square Tower 2, #02-10, 2 Marina View",ChIJxceAJL4Z2jERPzmznohK1m0,Afro Asia,1.278265,...,Izzy Teh,Expired,"Return Cashback, Offline Pay",None,87.916667,Gochi-So Shokudo,Yuping Goh,Created,None,None
1,OPERATIONAL,Avorush,4.0,170.0,"['cafe', 'food', 'point_of_interest', 'establi...",NaN,"#02-23 Asia Square Tower 2, 12 Marina View",ChIJU01vhBEZ2jERxS5dJlopTE4,Afro Asia,1.278177,...,Jonathan Tan,Expired,"Offline Pay, Return Cashback",None,100.0,Avorush,Jacqueline Lim,Created,None,None
2,OPERATIONAL,Subway,3.4,43.0,"['restaurant', 'food', 'point_of_interest', 'e...",2.0,"12 Marina View, #02-07, Asia Square Tower 2, S...",ChIJp98riREZ2jERF_EQ2FPd4RU,Afro Asia,1.278661,...,cassandra pek,Live,"Offline Pay, Return Cashback","Offline Pay, Return Cashback",100.0,Subway,Yolanda Tan,Live,"Return Cashback, Card Linked Offer, Offline Pa...","Return Cashback, Card Linked Offer, Offline Pay"
3,OPERATIONAL,Paninoteca,4.4,61.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View 01-04 Asia Square Tower, #2",ChIJbfG63x4Z2jERlAh3_XkI4ao,Afro Asia,1.278286,...,Hafiz Saleh,Live,Offline Pay + Return Cashback (Apr 2023),Offline Pay + Return Cashback (Apr 2023),100.0,Paninoteca,Product Payments,Created,None,None
4,OPERATIONAL,KPMG Clubhouse,4.6,128.0,"['restaurant', 'food', 'point_of_interest', 'e...",NaN,"12 Marina View, #15-00 Asia Square Tower 2",ChIJ_R4q2w0Z2jERtNBGtAqR5YY,Afro Asia,1.278265,...,Yuping Goh,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",59.710145,Clubhouse SG,Alavie Lee,Created,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30287,,,,,,,China Square,,,1.4187298,...,Automation Bot,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,,,,,,
30288,,,,,,,Velocity,,,1.4187298,...,Automation Bot,Expired,Offline Pay + Return Cashback (Apr 2023),None,,,,,,
30289,,,,,,,Pasir Ris West Plaza,,,1.4187298,...,Automation Bot,Expired,"Offline Pay + Return Cashback (Apr 2023), Card...",None,,,,,,
30290,,,,,,,23 Serangoon Central,,,1.4187298,...,Automation Bot,Live,"Offline Pay + Return Cashback (Apr 2023), Card...","Offline Pay + Return Cashback (Apr 2023), Card...",,,,,,


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6a158595-5c60-4034-bb24-659459b82f97' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>